<center><img src="http://i.imgur.com/sSaOozN.png" width="500"></center>


## Course: Computational Thinking for Governance Analytics

### Prof. José Manuel Magallanes, PhD 
* Visiting Professor of Computational Policy at Evans School of Public Policy and Governance, and eScience Institute Senior Data Science Fellow, University of Washington.
* Professor of Government and Political Methodology, Pontificia Universidad Católica del Perú. 

_____
<a id='home'></a>

# Introduction to Optimization


In [65]:
%%html
<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vTSzx6Vjo9on9XRNlPgdkxQszdpYZgYEQZYrlf3EYca7Fvze0sui8LphkWlTtscq-zQMBH5ZuyS_PhY/embed?start=false&loop=false&delayms=3000" frameborder="0" width="900" height="600" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

Please, go to your _environment_ in Anacoda Navigator to install **glpk** and **pulp**  before runing the codes below.

In [2]:
from pulp import LpMaximize,LpProblem,LpVariable,LpConstraint, value
from pulp import COIN_CMD
from pulp import LpConstraintGE as GE
from pulp import LpConstraintLE as LE


model = LpProblem(name='refinery-problem', sense=LpMaximize)

gas = LpVariable(name="Gas", lowBound=0,cat='Continuous')
oil = LpVariable(name="Oil", lowBound=0,cat='Continuous')

obj_func = 1.9*gas + 1.5*oil

C1= LpConstraint(name='Gas Constraint', 
                 e= 1*gas - 2*oil, sense=GE, rhs=0)
C2= LpConstraint(name='Oil Constraint',
                 e= 1*oil, sense=GE, rhs=3000000)
C3= LpConstraint(name='Demand Constraint',
                 e= 1*gas, sense=LE, rhs=6400000)


# build model with info and solve the model

model += obj_func
model += C1
model += C2
model += C3

#solver = COIN_CMD(msg=False)
result=model.solve()#solver)

Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Apr 19 2023 

command line - cbc /var/folders/2n/bkfhfqq16r78g3hf7pdj56y40000gn/T/895a8acbc1ac4d619f9e20fcc232fdd1-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/2n/bkfhfqq16r78g3hf7pdj56y40000gn/T/895a8acbc1ac4d619f9e20fcc232fdd1-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 15 RHS
At line 19 BOUNDS
At line 20 ENDATA
Problem MODEL has 3 rows, 2 columns and 4 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-3) rows, 0 (-2) columns and 0 (-4) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 16960000
After Postsolve, objective 16960000, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 16960000 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seco

In [3]:
print ("Optimal Result:")
for variable in model.variables():
    print (variable.name, "=", variable.varValue)

Optimal Result:
Gas = 6400000.0
Oil = 3200000.0


In [4]:
print ("Total net max profit:")
print (value(model.objective))

Total net max profit:
16960000.0


In [5]:
print ("Slack/Surplus")
for name, constraint in model.constraints.items():
    print(name + ':' + str(constraint.value()))

Slack/Surplus
Gas_Constraint:0.0
Oil_Constraint:200000.0
Demand_Constraint:0.0


In [7]:
import pandas as pd

o = [{'name':name,'shadow price':c.pi,'slack': c.slack} for name, c in model.constraints.items()]

print(pd.DataFrame(o))

                name  shadow price     slack
0     Gas_Constraint         -0.75      -0.0
1     Oil_Constraint         -0.00 -200000.0
2  Demand_Constraint          2.65      -0.0


In [66]:
%%html

<iframe src="https://docs.google.com/presentation/d/e/2PACX-1vTjLUWEMZLMVTv2Yni9GPgY4XddrJEFVx-9WDsnD2jhGKeLPteQrkH0xMRm97iE01pK0OozDkksC2Cw/embed?start=false&loop=false&delayms=3000" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>


In [61]:
from pulp import LpMinimize

model = LpProblem(name='diet-problem', sense=LpMinimize)

V_V = LpVariable(name="Vega Vita", lowBound=0,cat='Integer')
H_H = LpVariable(name="Happy Health", lowBound=0,cat='Integer')

obj_func = 0.2*V_V + 0.3*H_H

C1=LpConstraint(e=20*V_V + 30*H_H,sense=GE,name='Vitamin C',rhs=60)
C2=LpConstraint(e=500*V_V + 250*H_H,sense=GE,name='Calcium',rhs=1000)
C3=LpConstraint(e=9*V_V + 2*H_H,sense=GE,name='Iron',rhs=18)
C4=LpConstraint(e=2*V_V + 10*H_H,sense=GE,name='Niacin',rhs=20)
C5=LpConstraint(e=60*V_V + 90*H_H,sense=GE,name='Magnesium',rhs=360)


model += obj_func
model += C1
model += C2
model += C3
model += C4
model += C5

solver = COIN_CMD(msg=False)
result=model.solve(solver)

In [62]:
print ("Optimal Result:")
for variable in model.variables():
    print (variable.name, "=", variable.varValue)

Optimal Result:
Happy_Health = 2.0
Vega_Vita = 3.0


In [63]:
print ("Total min expenses:")
print (value(model.objective))

Total min expenses:
1.2000000000000002


In [64]:
print ("Slack/Surplus")
for name, constraint in model.constraints.items():
    print(name + ':' + str(constraint.value()))

Slack/Surplus
Vitamin_C:60.0
Calcium:1000.0
Iron:13.0
Niacin:6.0
Magnesium:0.0
